## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,54.05,74,100,11.45,overcast clouds
1,1,Bluff,NZ,-46.60,168.33,50.81,88,100,8.12,overcast clouds
2,2,Batemans Bay,AU,-35.72,150.18,51.10,87,100,6.51,overcast clouds
3,3,Nurota,UZ,40.56,65.69,39.29,41,0,3.53,clear sky
4,4,Cumra,TR,37.57,32.77,64.54,22,0,2.53,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Cumra,TR,37.57,32.77,64.54,22,0,2.53,clear sky
6,6,Coquimbo,CL,-29.95,-71.34,65.93,39,0,14.97,clear sky
9,9,Naqadeh,IR,36.96,45.39,55.36,47,0,1.79,clear sky
10,10,Hermanus,ZA,-34.42,19.23,65.89,69,94,9.01,overcast clouds
12,12,Rikitea,PF,-23.12,-134.97,74.71,80,91,18.59,light rain
13,13,Bredasdorp,ZA,-34.53,20.04,63.28,77,9,13.80,clear sky
18,18,Henties Bay,NaN,-22.12,14.28,60.85,65,0,2.66,clear sky
20,20,Chicama,PE,-7.84,-79.15,59.54,63,39,10.56,scattered clouds
21,21,Lebu,CL,-37.62,-73.65,56.19,70,26,8.77,scattered clouds
24,24,Saldanha,ZA,-33.01,17.94,64.45,63,100,10.33,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Cumra,TR,64.54,clear sky,37.57,32.77,
6,Coquimbo,CL,65.93,clear sky,-29.95,-71.34,
9,Naqadeh,IR,55.36,clear sky,36.96,45.39,
10,Hermanus,ZA,65.89,overcast clouds,-34.42,19.23,
12,Rikitea,PF,74.71,light rain,-23.12,-134.97,
13,Bredasdorp,ZA,63.28,clear sky,-34.53,20.04,
20,Chicama,PE,59.54,scattered clouds,-7.84,-79.15,
21,Lebu,CL,56.19,scattered clouds,-37.62,-73.65,
24,Saldanha,ZA,64.45,overcast clouds,-33.01,17.94,
30,Jamestown,US,71.19,scattered clouds,42.10,-79.24,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# Empty strings present in the hotel names, so convert them to NaNs so we can drop them
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace(r'^\s*$', np.NaN, regex=True)

In [19]:
hotel_df.count()

City                   200
Country                200
Max Temp               200
Current Description    200
Lat                    200
Lng                    200
Hotel Name             190
dtype: int64

In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
len(clean_hotel_df)

190

In [21]:
clean_hotel_df.count()

City                   190
Country                190
Max Temp               190
Current Description    190
Lat                    190
Lng                    190
Hotel Name             190
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))